## Conduct vector similarity search on Azure OpenAI embeddings using Azure Managed Redis

- Tutorial: https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/cache-tutorial-vector-similarity
- Code: https://github.com/Azure-Samples/azure-cache-redis-samples/tree/main/tutorial/vector-similarity-search-open-ai

### Install dependencies
Install the python dependencies required for our application. Using a Python virtual environment is usually a good idea.

In [ ]:
# Code cell 1

! pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken redis langchain langchain_openai langchain_community langchain-redis
! pip install langchain-huggingface sentence-transformers scikit-learn
# ! pip install -r requirements.txt

### Import libraries and set up Azure OpenAI and Azure Managed Redis connection info
Fill in your Azure OpenAI and Azure Managed Redis information below. This will be used later to establish the connection these services, generate the embeddings, and load them into Redis. This example stores these values in application variables for the sake of simplicity. Outside of tutorials, it's strongly recommended to store these in environment variables or using a secrets manager like Azure KeyVault. 

Note that there are differences  between the `OpenAI` and `Azure OpenAI` endpoints. This example uses the configuration for `Azure OpenAI`. See [How to switch between OpenAI and Azure OpenAI endpoints with Python](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/switching-endpoints) for more details. 

In [ ]:
# Code cell 2
import re
import os
import pandas as pd
import tiktoken
from typing import List
from dotenv import load_dotenv
from num2words import num2words
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Redis as RedisVectorStore
from langchain.document_loaders import DataFrameLoader

load_dotenv()

API_KEY = os.getenv('API_KEY')
RESOURCE_ENDPOINT = os.getenv('RESOURCE_ENDPOINT')
DEPLOYMENT_NAME = os.getenv('DEPLOYMENT_NAME')
MODEL_NAME = os.getenv('MODEL_NAME')
REDIS_ENDPOINT = os.getenv('REDIS_ENDPOINT')
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')

print(f"RESOURCE_ENDPOINT: {RESOURCE_ENDPOINT}")
print(f"REDIS_ENDPOINT: {REDIS_ENDPOINT}")
print(f"DEPLOYMENT_NAME: {DEPLOYMENT_NAME}")
print(f"MODEL_NAME: {MODEL_NAME}")


### Import dataset

This example uses the [Wikipedia Movie Plots](https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots) dataset from Kaggle. Download this file and place it in the same directory as this jupyter notebook.  

In [ ]:
# Code cell 3
df=pd.read_csv(os.path.join(os.getcwd(),'wiki_movie_plots_deduped.csv'))
df

Process the dataset to remove spaces in the column titles and filter the dataset to lower the size. This isn't required, but is helpful in reducing the time it takes to generate embeddings and loading the index into Redis. Feel free to play around with the filters, or add your own! 

In [ ]:
# Code cell 4

df.insert(0, 'id', range(0, len(df)))
df['year'] = df['Release Year'].astype(int)
df['origin'] = df['Origin/Ethnicity'].astype(str)
del df['Release Year']
del df['Origin/Ethnicity']
df = df[df.year > 1970] # only movies made after 1970
df = df[df.origin.isin(['American','British','Canadian'])] # only movies from English-speaking cinema
df

Remove whitespace from the `Plot` column to make it easier to generate embeddings.

In [ ]:
# Code cell 5

pd.options.mode.chained_assignment = None

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['Plot']= df['Plot'].apply(lambda x : normalize_text(x))

Calculate the number of tokens required to generate the embeddings for this dataset. You may want to filter the dataset more stringently in order to limit the tokens required. 

In [ ]:
# Code cell 6

tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["Plot"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
print('Number of movies: ' + str(len(df))) # print number of movies remaining in dataset
print('Number of tokens required:' + str(df['n_tokens'].sum())) # print number of tokens

### Load Dataframe into LangChain
Using the `DataFrameLoader` class allows you to load a pandas dataframe into LangChain. That makes it easy to load your data and use it to generate embeddings using LangChain's other integrations.

In [ ]:
# Code cell 7

loader = DataFrameLoader(df, page_content_column="Plot" )
movie_list = loader.load()

### Generate embeddings and Load them into Azure Managed Redis
Using LangChain, this example connects to Azure OpenAI Service to generate embeddings for the dataset. These embeddings are then loaded into [Azure Managed Redis](https://learn.microsoft.com/en-us/azure/azure-cache-for-redis/managed-redis/managed-redis-overview), a fully managed Redis service on Azure, which features the [RediSearch](https://redis.io/docs/latest/develop/interact/search-and-query/) module that includes vector search capability. Finally, a copy of the index schema is saved. That is useful for loading the index into Redis later if you don't want to regenerate the embeddings.

In [ ]:
# Code cell 8

# we will use Azure OpenAI as our embeddings provider
embedding = AzureOpenAIEmbeddings(
    azure_endpoint=RESOURCE_ENDPOINT,
    azure_deployment=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_version='2024-03-01-preview',
    show_progress_bar=True,
    chunk_size=16)

# name of the Redis search index to create
index_name = "movieindex"

# create a connection string for the Redis Vector Store. Uses Redis-py format: https://redis-py.readthedocs.io/en/stable/connections.html#redis.Redis.from_url
# This example assumes TLS is enabled. If not, use "redis://" instead of "rediss://
redis_url = "rediss://:" + REDIS_PASSWORD + "@"+ REDIS_ENDPOINT

# Take the first 100 movies
# short_list = movie_list[:100]

# create and load redis with documents
vectorstore = RedisVectorStore.from_documents(
    documents=movie_list,
    embedding=embedding,
    index_name=index_name,
    redis_url=redis_url
)

# save index schema so you can reload in the future without re-generating embeddings
vectorstore.write_schema("redis_schema.yaml")

# This may take up to 10 minutes to complete.

### Run search queries
Using the vectorstore we just built in LangChain, we can conduct similarity searches using the `similarity_search_with_score` method. In this example, the top 10 results for a given query are returned.

In [ ]:
# Code cell 9

results = vectorstore.similarity_search_with_score(query="Spaceships, aliens, and heroes saving America", k=10)

for doc, score  in enumerate(results):
    movie_title = str(results[doc][0].metadata['Title'])
    similarity_score = str(round((1 - results[doc][1]),4))
    print(movie_title + ' (Score: ' + similarity_score + ')')


### Run hybrid queries

You can also run hybrid queries. That is, queries that use both vector search and filters based on other parameters in the dataset. In this case, we filter our query results to only movies tagged with the `comedy` genre. One of the advantages of using LangChain with Redis is that metadata is preserved in the index, so you can use it to filter your results. 

In [ ]:
# Code cell 10

from langchain.vectorstores.redis import RedisText

query = "Spaceships, aliens, and heroes saving America"
genre = "comedy"

genre_filter = RedisText("Genre") == genre

results = vectorstore.similarity_search_with_score(query, filter=genre_filter, k=10)
for i, j in enumerate(results):
    movie_title = str(results[i][0].metadata['Title'])
    similarity_score = str(round((1 - results[i][1]),4))
    print(movie_title + ' (Score: ' + similarity_score + ')')

### Appendix A: Load index data already in Redis
If you already have embeddings data in Redis, you can load it into your LangChain vectorstore oboject using the `from_existing_index` method. This is useful if you don't want to re-run your embeddings model. You'll need to provide the index schema that was saved when you generated the embeddings.

In [ ]:
# Code cell 11

# we will use Azure OpenAI as our embeddings provider
embedding = AzureOpenAIEmbeddings(
    azure_endpoint=RESOURCE_ENDPOINT,
    azure_deployment=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_version='2024-03-01-preview',
    show_progress_bar=True,
    chunk_size=16)

# name of the Redis search index to create
index_name = "movieindex"

# create a connection string for the Redis Vector Store. Uses Redis-py format: https://redis-py.readthedocs.io/en/stable/connections.html#redis.Redis.from_url
# This example assumes TLS is enabled. If not, use "redis://" instead of "rediss://
redis_url = "rediss://:" + REDIS_PASSWORD + "@"+ REDIS_ENDPOINT

vectorstore = RedisVectorStore.from_existing_index(
    embedding=embedding,
    redis_url=redis_url,
    index_name=index_name,
    schema="redis_schema.yaml"
)

### Appendix B: Query Redis using the CLI

In [ ]:
# Install RedisVL
! pip install redisvl

In [ ]:
# Query Azure Manager Redis
! rvl index listall -u $redis_url
! rvl index info -i movieindex -u $redis_url
! rvl stats -i movieindex -u $redis_url


In [ ]:
# Destroy movie index
! rvl index destroy -i movieindex -u $redis_url

### Appendix C: Simple RAG Chain (no chat hostory or question rewriting)

See `movie-chat.py` and `movie-chat-ui.py` for chat history and question rewriting (history aware retriever usage).

```sh
python movie-chat.py
# or
streamlit run movie-chat-ui.py
```



In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("RESOURCE_ENDPOINT"),
    azure_deployment='gpt-4o-mini',
    api_key=os.getenv("API_KEY"),
    openai_api_version="2024-09-01-preview"
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """You are a movie buff who can answer questions about movies, make suggestions, summarise key facts, and provide other useful movie information. Use the following information as context to build your answer. If you are unsure, just say 'I'm unsure".  Only discuss movies from the context provided.  Don't discuss other topics not related to the movies.
Question: {question} 
Context: {context} 
Answer:""",
        ),
    ]
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

while True:
    question = input("What's your question about movie(s)? ")
    if question == 'q' or question == '':
        print('Bye!')
        break
    else:
        answer = rag_chain.invoke(question)

        print(f'\nAnswer:\n{answer}')
